In [1]:
import os
import torch
os.environ["VISIBLE_CUDA_DEVICES"]="0"
os.environ["WANDB_API_KEY"] = "007c52154146420e5b9b166408c9eeef9eb20599"

In [2]:
import torch
from torch.utils.data import Subset,DataLoader
from transformers import TrainingArguments, Trainer,BlipProcessor, BlipForQuestionAnswering
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset,load_dataset
from PIL import Image
import pandas as pd
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

2025-05-15 14:53:10.947767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747320791.172447      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747320791.234681      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from torch.utils.data import Dataset
# from datasets import Dataset
from PIL import Image
import os
class VQADataset(Dataset):
    def __init__(self, data, processor):
        """
        Args:
            data (list of dicts): Each item should have keys: 'image', 'question', 'answer'
            processor: BLIP-2 processor (e.g., Blip2Processor.from_pretrained(...))
            image_root_dir (str): Root directory where images are stored
        """
        # super().__init__()
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        try:
            # Retrieve the data item for the current index
            item = self.data.iloc[idx]
    
            # Construct the full path to the image
            image_path = os.path.join("/kaggle/input/vrdataset", item['image_path'])
    
            # Check if the image exists
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found: {image_path}")
    
            # Try to open the image and convert to RGB
            try:
                image = Image.open(image_path).convert("RGB")
            except Exception as img_err:
                print(f"[ERROR] Unable to open image at index {idx}: {image_path}, Error: {str(img_err)}")
                # You can choose to raise or skip based on your use-case
                raise img_err
    
            # Extract question and answer
            question = str(item["question"])
            answer = str(item["response"])
    
            # Process the image and question using the BLIP-2 processor
            inputs = self.processor(
                images=image,
                text=question,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=128
            )
    
            # Remove batch dimension from the processed inputs
            inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    
            inputs["labels"] = processor.tokenizer(answer, return_tensors="pt", padding='max_length', max_length=32, truncation=True).input_ids.squeeze(0)

    
            # print(f"Returning keys from dataset{idx}: {inputs.keys()}")
            # if idx==18888:print(image)
            return inputs
    
        except FileNotFoundError as e:
            print(f"[ERROR] FileNotFoundError: {str(e)}")
            raise e
        except KeyError as e:
            print(f"[ERROR] KeyError: Missing key in data at index {idx}: {str(e)}")
            raise e
        except Exception as e:
            print(f"[ERROR] Unexpected error at index {idx}: {str(e)}")
            raise e

In [4]:
# train_df =  pd.read_csv("/kaggle/input/train-test-val/"+"/train.csv")
train_df = pd.read_csv('/kaggle/input/60k-train-dataset/train_dataset.csv')
val_df = pd.read_csv("/kaggle/input/train-test-val/"+"/val.csv")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
train_dataset = VQADataset(train_df,processor)
val_dataset = VQADataset(val_df,processor)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
print(len(train_dataset))

67001


In [6]:
sample_item = train_dataset[0]
print(sample_item.keys())  # Should include 'input_ids', 'attention_mask', 'pixel_values', 'labels'
print(sample_item['input_ids'].shape)
print(sample_item['labels'].shape)
print(sample_item['pixel_values'].shape)
print( sample_item['attention_mask'].shape)
print(sample_item)

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'labels'])
torch.Size([128])
torch.Size([32])
torch.Size([3, 384, 384])
torch.Size([128])
{'pixel_values': tensor([[[1.9011, 1.9011, 1.9011,  ..., 1.9157, 1.9157, 1.9157],
         [1.9011, 1.9011, 1.9011,  ..., 1.9157, 1.9157, 1.9157],
         [1.9011, 1.9011, 1.9011,  ..., 1.9157, 1.9157, 1.9157],
         ...,
         [1.8573, 1.8573, 1.8573,  ..., 1.8573, 1.8573, 1.8573],
         [1.8573, 1.8573, 1.8573,  ..., 1.8573, 1.8573, 1.8573],
         [1.8573, 1.8573, 1.8573,  ..., 1.8573, 1.8573, 1.8573]],

        [[2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599],
         [2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599],
         [2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599],
         ...,
         [2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599],
         [2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599],
         [2.0749, 2.0749, 2.0749,  ..., 2.0599, 2.0599, 2.0599]],

        [[2.1317, 2.131

In [7]:
sample_item = val_dataset[0]
print(sample_item.keys())  # Should include 'input_ids', 'attention_mask', 'pixel_values', 'labels'
print(sample_item['input_ids'].shape)
print(sample_item['labels'].shape)
print(sample_item['pixel_values'].shape)
print( sample_item['attention_mask'].shape)
print(sample_item)

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'labels'])
torch.Size([128])
torch.Size([32])
torch.Size([3, 384, 384])
torch.Size([128])
{'pixel_values': tensor([[[1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157],
         [1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157],
         [1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157],
         ...,
         [1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157],
         [1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157],
         [1.9157, 1.9157, 1.9157,  ..., 1.9157, 1.9157, 1.9157]],

        [[2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599],
         [2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599],
         [2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599],
         ...,
         [2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599],
         [2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599],
         [2.0599, 2.0599, 2.0599,  ..., 2.0599, 2.0599, 2.0599]],

        [[2.1317, 2.131

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model in float16
model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base")


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [9]:
def collate_fn(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch], batch_first=True, padding_value=-100
    )
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    # target_modules=["q_proj","v_proj"],  # Target attention layers
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    # task_type=TaskType.CAUSAL_LM  # or
)
# # Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 387,031,868 || trainable%: 0.6096


In [11]:

from transformers import DefaultDataCollator,PrinterCallback,EarlyStoppingCallback,ProgressCallback

args = TrainingArguments(
    output_dir="/kaggle/working/output_dir",
    eval_strategy="epoch",
    run_name="blipvqabase-full-16-ranklora-siddhesh",
    gradient_accumulation_steps=1,
    num_train_epochs=8,
    fp16=True,
    warmup_ratio=0.1,
    label_names=["labels"],
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    save_strategy="epoch",
    logging_strategy="epoch",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    lr_scheduler_type="cosine",
    learning_rate=1e-4
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2,early_stopping_threshold=0.001),PrinterCallback(),ProgressCallback()]
)

In [12]:
print("Training Started")
trainer.train()

Training Started


wandb: Currently logged in as: siddhesh-deshpande (siddhesh-deshpande-international-institute-of-informatio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250515_145342-vjsn41yn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run blipvqabase-full-16-ranklora-siddhesh
wandb: ⭐️ View project at https://wandb.ai/siddhesh-deshpande-international-institute-of-informatio/huggingface
wandb: 🚀 View run at https://wandb.ai/siddhesh-deshpande-international-institute-of-informatio/huggingface/runs/vjsn41yn


  0%|          | 0/8376 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch,Training Loss,Validation Loss
1,8.630000,8.163735
2,8.134700,8.125113
3,8.109300,8.112928
4,8.098000,8.106026
5,8.090500,8.100841
6,8.085600,8.098443
7,8.082500,8.097235
8,8.081300,8.097055


...

  0%|          | 0/159 [00:00<?, ?it/s]

{'eval_loss': 8.097234725952148, 'eval_runtime': 455.1883, 'eval_samples_per_second': 22.283, 'eval_steps_per_second': 0.349, 'epoch': 7.0}
{'eval_loss': 8.097234725952148, 'eval_runtime': 455.1883, 'eval_samples_per_second': 22.283, 'eval_steps_per_second': 0.349, 'epoch': 7.0}
{'loss': 8.0813, 'grad_norm': 17649.548828125, 'learning_rate': 4.342376558330585e-12, 'epoch': 8.0}
{'loss': 8.0813, 'grad_norm': 17649.548828125, 'learning_rate': 4.342376558330585e-12, 'epoch': 8.0}


  0%|          | 0/159 [00:00<?, ?it/s]

{'eval_loss': 8.097055435180664, 'eval_runtime': 452.975, 'eval_samples_per_second': 22.392, 'eval_steps_per_second': 0.351, 'epoch': 8.0}
{'eval_loss': 8.097055435180664, 'eval_runtime': 452.975, 'eval_samples_per_second': 22.392, 'eval_steps_per_second': 0.351, 'epoch': 8.0}
{'train_runtime': 33916.7636, 'train_samples_per_second': 15.804, 'train_steps_per_second': 0.247, 'train_loss': 8.163984734098914, 'epoch': 8.0}
{'train_runtime': 33916.7636, 'train_samples_per_second': 15.804, 'train_steps_per_second': 0.247, 'train_loss': 8.163984734098914, 'epoch': 8.0}


TrainOutput(global_step=8376, training_loss=8.163984734098914, metrics={'train_runtime': 33916.7636, 'train_samples_per_second': 15.804, 'train_steps_per_second': 0.247, 'train_loss': 8.163984734098914, 'epoch': 8.0})

In [13]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# After training…
output_dir = "/kaggle/working/weights"

# 1) Save the model weights & config
model.save_pretrained(output_dir)

# 2) (If you used a processor for images + text)
processor.save_pretrained(output_dir)  

[]

In [14]:
!zip -r weights.zip "/kaggle/working/weights"

  adding: kaggle/working/weights/ (stored 0%)
  adding: kaggle/working/weights/tokenizer_config.json (deflated 74%)
  adding: kaggle/working/weights/tokenizer.json (deflated 71%)
  adding: kaggle/working/weights/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/weights/vocab.txt (deflated 53%)
  adding: kaggle/working/weights/adapter_config.json (deflated 54%)
  adding: kaggle/working/weights/README.md (deflated 66%)
  adding: kaggle/working/weights/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/weights/preprocessor_config.json (deflated 49%)
